# Preparing NISAR data for validation of Solid Earth requirements

**Prepared by:** Emre Havazli in October 2025 replacing the initial notebook prepared by Ekaterina Tymofyeyeva, Heresh Fattahi, Sara Mirzaee, Max Zhan, and Jeff Pon March 2024<br>

<div class="alert alert-warning">
This notebook pre-processes data for different NISAR Solid Earth calval sites amd requirements. Subsequent validation is done via separate notebooks for the Transient, Secular, and Coseismic requirements. These are located under /ATBD_main/methods/.
</div>

<hr/>

## Table of Contents: <a id='prep_TOC'></a>

[**Environment Setup**](#setup)
- [Load Python Packages](#load_packages)
- [Define CalVal Site and Parameters](#set_calval_params)
- [Define Directories](#set_directories)
- [Authentication](#set_authentication)

[**1. Download and Prepare Interferograms**](#prep_ifg)
- [1.1.  Get Data List](#prep_data)
- [1.2.  Download DEM](#prep_dem)
- [1.3.  Create MintPy configuration file](#create_config)
- [1.4.  Load Data into MintPy](#prep_load_data)

<hr/>

<a id='#setup'></a>
## Environment Setup

### Load Python Packages <a id='#load_packages'></a>

In [ ]:
import json
import netrc
import os
import subprocess
import glob

import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_origin
from pyproj import Transformer
import tile_mate

### Define Calval Site and Parameters <a id='set_calval_params'></a>

In [ ]:
# === Basic Configuration ===
site = "Erta_Ale"  # Cal/Val location ID
requirement = "Transient"  # Options: 'Secular', 'Coseismic', 'Transient'
dataset = "NISAR_sample"  # Dataset type: 'NISAR_sample', 'ARIA_S1', 'ARIA_S1_new'


rundate = "20260205"  # Date of this Cal/Val run
version = "1"         # Version of this Cal/Val run
custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata

# === Username Detection / Creation ===
user_file = "/home/jovyan/me.txt"
if os.path.exists(user_file):
    with open(user_file, "r") as f:
        you = f.readline().strip()
else:
    you = input("Please type a username for your Cal/Val outputs: ").strip()
    with open(user_file, "w") as f:
        f.write(you)

# === Load Cal/Val Site Metadata ===
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

print(f"Loaded site: {site}")

### Set Directories and Files <a id='set_directories'></a>

In [ ]:
# Static base directory for Cal/Val processing
BASE_DIR = "/scratch/nisar-st-calval-solidearth"

# Define key path components
site_dir = os.path.join(BASE_DIR, dataset, site)
work_dir = os.path.join(site_dir, requirement, you, rundate, f"v{version}")
gunw_dir = os.path.join(site_dir, "products")
mintpy_dir = os.path.join(work_dir, "MintPy")
dem_dir = os.path.join(work_dir, "DEM")

# Create required directories
for path in [work_dir, gunw_dir, mintpy_dir, dem_dir]:
    os.makedirs(path, exist_ok=True)

# Set working directory
os.chdir(work_dir)

# Log directory structure
print(f"  Work directory: {work_dir}")
print(f"  GUNW directory: {gunw_dir}")
print(f"MintPy directory: {mintpy_dir}")
print(f"DEM directory: {dem_dir}")

# Configuration file path
site_code = site_info.get('calval_location')
config_file = os.path.join(mintpy_dir, f"{site_code}.cfg")

### Authentication <a id='set_authentication'></a>

In [ ]:
def ensure_permission(path, mode=0o600):
    if os.path.exists(path):
        os.chmod(path, mode)

# === Earthdata Login ===
fnetrc = os.path.expanduser("~/.netrc")
earthdata_host = "urs.earthdata.nasa.gov"
earthdata = False

if os.path.exists(fnetrc):
    ensure_permission(fnetrc)
    nrc = netrc.netrc()
    credentials = nrc.authenticators(earthdata_host)
    if credentials:
        earthdata_user, _, earthdata_password = credentials
        earthdata = True
        print(f"Earthdata credentials found for user: {earthdata_user}")

if not earthdata:
    print("\nNEEDED to Download ARIA GUNWs")
    print("Create account at: https://urs.earthdata.nasa.gov/")
    earthdata_user = input("Earthdata username: ").strip()
    earthdata_password = input("Earthdata password: ").strip()
    with open(fnetrc, "a") as f:
        f.write(f"machine {earthdata_host}\nlogin {earthdata_user}\npassword {earthdata_password}\n")
    ensure_permission(fnetrc)
    print("Earthdata credentials saved.")

# === OpenTopography API Key ===
fopentopo = os.path.expanduser("~/.topoapi")
if os.path.exists(fopentopo):
    ensure_permission(fopentopo)
    with open(fopentopo) as f:
        opentopography_api_key = f.read().strip()
else:
    print("\nNEEDED To Download DEMs:")
    print("Register at: https://portal.opentopography.org/login")
    print("Navigate: My Account → myOpenTopo Authorizations and API Key → Request API key")
    opentopography_api_key = input("OpenTopo API key: ").strip()
    with open(fopentopo, "w") as f:
        f.write(opentopography_api_key + "\n")
    ensure_permission(fopentopo)
    print("OpenTopography API key saved.")

# === CDS (ERA5) API Key ===
cds_config_path = os.path.expanduser("~/.cdsapirc")
if not os.path.exists(cds_config_path):
    print("\nNEEDED to use ERA5 correction:")
    print("Register and get token: https://cds.climate.copernicus.eu/how-to-api")
    cds_key = input("CDS API key (uid:api-key): ").strip()
    with open(cds_config_path, "w") as f:
        f.write("url: https://cds.climate.copernicus.eu/api\n")
        f.write(f"key: {cds_key}\n")
    ensure_permission(cds_config_path)
    print("CDS API config created.")
else:
    print("CDS API config file detected. (Ensure it is current)")

<br>
<hr>

<a id='prep_ifg'></a>
## 1. Download and Prepare Interferograms

In this initial processing step, all the necessary Level-2 unwrapped interferogram products are gathered and organized for analysis with MintPy.

### 1.1. Get data list <a id='prep_data'></a>

In [ ]:
# Parse date range from site metadata
start_date = int(site_info.get('download_start_date'))
end_date = int(site_info.get('download_end_date'))

# Filter GUNW files based on date range and version
gunw_list = []
for filename in os.listdir(gunw_dir):
    if filename.endswith(".h5"):
        ref_date = int(filename.split('_')[12].split('T')[0])
        sec_date = int(filename.split('_')[13].split('T')[0])
        if start_date <= ref_date and sec_date <= end_date:
            gunw_list.append(os.path.join(gunw_dir, filename))
        else:
            print(f"Warning: Skipping malformed filename: {filename}")         
    else:
        continue

# Sort and write list to product file for future use with ARIA-tools
gunw_list.sort()
product_file = os.path.join(work_dir, "product_file.txt")
with open(product_file, "w") as f:
    f.write("\n".join(gunw_list))

print(f"Wrote {len(gunw_list)} GUNW files to: {product_file}")


### 1.2. Download DEM <a id='prep_dem'></a>

In [ ]:
# Parse "lat_min lat_max lon_min lon_max" (e.g., "18.50 20.50 -156 -154")
region_str = site_info.get('analysis_region').strip("'\"")
lat_min, lat_max, lon_min, lon_max = map(float, region_str.split())

# Reorder to left,bottom,right,top for --bbox
bbox = [str(lon_min), str(lat_min), str(lon_max), str(lat_max)]

out_dem = os.path.join(dem_dir, f"{site}_elevation.dem")

if os.path.isfile(out_dem):
    print(f"{out_dem} exists, skip downloading")
else:
    cmd = [
        "sardem",
        "--bbox", *bbox,
        "--output", out_dem,
        "--output-format", "ENVI"
    ]
    
    print("Running:", " ".join(cmd))
    try:
        res = subprocess.run(cmd, check=True, capture_output=True, text=True)
        if res.stdout:
            print(res.stdout)
        print("DEM files:", os.listdir(dem_dir))
    except subprocess.CalledProcessError as e:
        print("sardem failed:")
        print(e.stdout or "")
        print(e.stderr or "")
        raise

## 1.3. Download Water Mask

In [ ]:
out_mask = f"{work_dir}/waterMask.msk"   # GeoTIFF file, just named .msk as requested
res_m = 40                   # WorldCover is ~10m; NISAR is ~80m
dst_epsg = 32611              # output CRS

# -----------------------
# 1) Download WorldCover for bounds (in lon/lat)
# -----------------------
bounds_ll = (lon_min, lat_min, lon_max, lat_max)
src_arr, src_prof = tile_mate.get_raster_from_tiles(bounds_ll, tile_shortname="esa_world_cover_2021")

# WorldCover water class = 80 -> 0; everything else -> 1
src_arr = np.asarray(src_arr)
src_mask = np.where(src_arr == 80, 0, 1).astype("uint8")

# -----------------------
# 2) Build destination grid in EPSG:32611 at res_m
# -----------------------
to_utm = Transformer.from_crs("EPSG:4326", f"EPSG:{dst_epsg}", always_xy=True)

# project the 4 corners, then take min/max
xs, ys = [], []
for lon, lat in [(lon_min, lat_min), (lon_min, lat_max), (lon_max, lat_min), (lon_max, lat_max)]:
    x, y = to_utm.transform(lon, lat)
    xs.append(x); ys.append(y)

xmin, xmax = min(xs), max(xs)
ymin, ymax = min(ys), max(ys)

width  = int(np.ceil((xmax - xmin) / res_m))
height = int(np.ceil((ymax - ymin) / res_m))

dst_transform = from_origin(xmin, ymax, res_m, res_m)  # top-left = (xmin, ymax)
dst_arr = np.zeros((height, width), dtype="uint8")

print("Projected bounds (UTM):", xmin, ymin, xmax, ymax)
print("Output shape:", height, width)

# -----------------------
# 3) Reproject mask onto destination grid and write GeoTIFF
# -----------------------
reproject(
    source=src_mask,
    destination=dst_arr,
    src_transform=src_prof["transform"],
    src_crs=src_prof["crs"],
    dst_transform=dst_transform,
    dst_crs=f"EPSG:{dst_epsg}",
    resampling=Resampling.nearest,
    src_nodata=0,
    dst_nodata=0,
)

with rasterio.open(
    out_mask,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype="uint8",
    crs=f"EPSG:{dst_epsg}",
    transform=dst_transform,
    nodata=0,
    compress="DEFLATE",
) as dst:
    dst.write(dst_arr, 1)

print(f"Wrote {out_mask}  (shape={dst_arr.shape}, crs=EPSG:{dst_epsg}, res={res_m}m)")

### 1.4. Create MintPy configuration file <a id='create_config'></a>

In [ ]:
os.chdir(mintpy_dir)

# Build config as a dictionary first
config_file_content = {
    "mintpy.load.processor": "nisar",
    "mintpy.compute.cluster": "local",
    "mintpy.compute.numWorker": "auto",
    "mintpy.load.waterMaskFile": out_mask,
    "mintpy.topographicResidual.pixelwiseGeometry": "no",
    "mintpy.troposphericDelay.method": "no",
    "mintpy.topographicResidual": "no",
    "mintpy.network.tempBaseMax": site_info.get('tempBaseMax'),
    "mintpy.network.startDate": site_info.get('download_start_date'),
    "mintpy.network.endDate": site_info.get('download_end_date'),
    "mintpy.velocity.startDate": site_info.get('download_start_date'),
    "mintpy.velocity.endDate": site_info.get('download_end_date'),
    "mintpy.reference.lalo": site_info.get('reference_lalo'),
    "mintpy.network.excludeDate12": site_info.get('ifgExcludePair'),
    "mintpy.network.excludeDate" : site_info.get('ifgExcludeDate'),
    "mintpy.network.excludeIfgIndex" : site_info.get('ifgExcludeIndex'),
}

# Write config dictionary to text file
with open(config_file, "w") as f:
    f.writelines(f"{k} = {v}\n" for k, v in config_file_content.items())

# Confirm output
print(f"MintPy config file written to:\n    {config_file}\n")
with open(config_file, "r") as f:
    print(f.read())


### 1.4. Load Data into MintPy Cubes <a id='prep_load_data'></a>

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing three HDF5 files:
- ifgramStack.h5: Contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata.
- geometryGeo.h5: Contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.).
- ionStack.h5   : Contains pairwise ionosphere data present in the NISAR L2 GUNW.
- tropoStack.h5 : Contains 3D interpolated total tropospheric delay correction calculated from NISAR radar grid tropospheric delay layers.
- setStack.h5   : Contains 3D interpolated solid Earth tide correction calculated from NISAR radar grid tropospheric delay layers.

In [ ]:
command = f'prep_nisar.py -i "{gunw_dir}/*.h5" -d {out_dem} -m {out_mask} -o {mintpy_dir}'
process = subprocess.run(command, shell=True)
print('Mintpy input files:')
[x for x in os.listdir(mintpy_dir + '/inputs') if x.endswith('.h5')]